In [14]:
import psycopg2
import pandas as pd

In [15]:
conn = psycopg2.connect(
            host='localhost',
            port='5432',
            database='airflow',
            user='postgres',
            password='MYZv3Ietuflo'
        )
cur = conn.cursor()

In [52]:
conn.rollback()

In [49]:
cur.execute(f'select * from weather')
result = cur.fetchall()
result

UndefinedTable: relation "weather" does not exist
LINE 1: select * from weather
                      ^


In [ ]:
conn.close()

In [1]:
with open(
    '/home/ivan_admin/dmls_aviaticket/ipynb/weather/weather_historical.csv'
    ) as file:
    csv = file.readlines()

In [20]:
len(csv)

10754

In [81]:
columns = [col for col in csv[0].replace('\n', '').split(',')[1:]]
types = ['timestamp'] + ['decimal(5, 1)'] * 6 + ['text'] + ['decimal(5, 1)'] * 12 + ['text'] * 2
table_name = 'weather'

In [82]:
col_type = ', '.join([col + ' ' + type for col, type in zip(columns, types)])
cur.execute(
    f'''
    create table if not exists {table_name} ({col_type});
    '''
)

cur.execute(f'select * from {table_name}')
result = cur.fetchall()
result

[]

In [83]:
table_data = []

for data in csv[1:]:
    data = data.replace('\n', '').split(',')[1:]
    data = ['NULL' if x == '' else x for x in data]
    string = ''
    for col, type in zip(data, types):
        if col == 'NULL':
            string += col + ', '
        elif type == 'text':
            string += "\'" + col + "\'" + ', '
        elif type == 'timestamp':
            string += 'TIMESTAMP ' + "\'" + col + "\'" + ', '
        else:
            col = col if col.replace('.', '').isdigit() else '0'
            string += col + ', '
    string = string[:-2]
    table_data.append(string)

columns = [col for col in csv[0].replace('\n', '').split(',')[1:]]
query = f"""insert into {table_name} ({', '.join(columns)}) \nvalues """
for row in table_data:
    query += f'\n({row}), '
query = query[:-2] + ';'

In [58]:
len(query)

1411030

In [84]:
print(query[:1000])

insert into weather (datetime, temp, feelslike, dew, humidity, precip, precipprob, preciptype, snow, snowdepth, windgust, windspeed, winddir, sealevelpressure, cloudcover, visibility, solarradiation, solarenergy, uvindex, severerisk, conditions, icon) 
values 
(TIMESTAMP '2021-11-01 00:00:00', 39.8, 34.5, 29.7, 66.84, 0.0, 0.0, NULL, 0.0, 0.0, NULL, 7.7, 260.0, 1022.3, 0.0, 9.9, NULL, NULL, NULL, NULL, 'Clear', 'clear-night'), 
(TIMESTAMP '2021-11-01 01:00:00', 39.0, 33.5, 28.9, 66.93, 0.0, 0.0, NULL, 0.0, 0.0, NULL, 7.8, 271.0, 1022.7, 0.0, 9.9, NULL, NULL, NULL, NULL, 'Clear', 'clear-night'), 
(TIMESTAMP '2021-11-01 02:00:00', 38.2, 32.9, 28.9, 68.76, 0.0, 0.0, NULL, 0.0, 0.0, NULL, 7.1, 281.0, 1022.8, 0.0, 9.9, NULL, NULL, NULL, NULL, 'Clear', 'clear-night'), 
(TIMESTAMP '2021-11-01 03:00:00', 37.3, 31.9, 28.9, 71.23, 0.0, 0.0, NULL, 0.0, 0.0, NULL, 7.0, 290.0, 1023.2, 0.0, 9.9, NULL, NULL, NULL, NULL, 'Clear', 'clear-night'), 
(TIMESTAMP '2021-11-01 04:00:00', 37.2, 31.9, 28.9, 71.

In [85]:
cur.execute(query)
conn.commit()

In [78]:
conn.rollback()

In [79]:
cur.execute('select * from weather;')
cur.fetchall()

[]

In [80]:
cur.execute('drop table weather;')

In [75]:
cur.execute('truncate table weather;')

In [76]:
conn.commit()